In [63]:
from datetime import datetime
startTime = datetime.now()

filepath = 'input.txt'

INSTRUCTION_JUMPS = {'01': 4, '02': 4, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4, '09': 2}
MODES = {'0': lambda i, n, data, relative_base: data[data[i+n]], 
         '1': lambda i, n, data, relative_base: data[i+n],
         '2': lambda i, n, data, relative_base: data[data[i+n] + relative_base]}

class intcode():
    def __init__(self, data):
        self.data = data + [0] * 1000
        self.i = 0
        self.num_inputs = 0
        self.relative_base = 0
        self.inputs = []
        self.internal_program = self.run_program()
        
    def add_input(self, addition):
        self.inputs.append(addition)
    
    def next_out(self):
        self.output = next(self.internal_program)
        return self.output
    
    def clear_inputs(self):
        self.inputs = []
        
    def run_program(self):
        while self.data[self.i] != 99:
            self.opcode = str(self.data[self.i])[-2:].zfill(2)
            self.mode = str(self.data[self.i])[:-2].zfill(3)

            if self.opcode in ('01', '02', '04', '05', '06', '07', '08', '09'):
                self.p1 = MODES[self.mode[2]](self.i, 1, self.data, self.relative_base)     
            if self.opcode in ('01', '02', '05', '06', '07', '08'):
                self.p2 = MODES[self.mode[1]](self.i, 2, self.data, self.relative_base)

            if self.opcode in ['03']:
                if self.mode[2] == '2':
                    self.p3 = self.data[self.i+1] + self.relative_base
                else:
                    self.p3 = self.data[self.i+1]
                self.data = self.data + [0] * (self.p3+1-len(self.data))                  

            if self.opcode in ['01', '02', '07', '08']:
                if self.mode[0] == '2':
                    self.p3 = self.data[self.i+3] + self.relative_base
                else:
                    self.p3 = self.data[self.i+3]
                self.data = self.data + [0] * (self.p3+1-len(self.data))

            if self.opcode == '01':
                self.data[self.p3] = self.p1 + self.p2
            elif self.opcode == '02':
                self.data[self.p3] = self.p1 * self.p2
            elif self.opcode == '03':
                #self.data[self.p3] = self.inputs[self.num_inputs]
                #print('taking input')
                
                if len(self.inputs) == 0:
                    yield None  
                self.data[self.p3] = self.inputs.pop(0)
                
                #self.num_inputs += 1
            elif self.opcode == '04':
                #print('outputting')
                #self.outputs.append(self.p1)
                yield self.p1 
            elif self.opcode == '05' and self.p1 != 0:
                self.i = self.p2
                continue
            elif self.opcode == '06' and self.p1 == 0:
                self.i = self.p2
                continue 
            elif self.opcode == '07':
                self.data[self.p3] = int(self.p1 < self.p2) 
            elif self.opcode == '08':
                self.data[self.p3] = int(self.p1 == self.p2)
            elif self.opcode == '09':
                self.relative_base += self.p1
            else:
                pass

            self.i += INSTRUCTION_JUMPS[self.opcode]
            
        #yield 'HALTED'
         
with open(filepath, 'r') as f:
    for line in f.readlines():
        data = [int(i) for i in line.strip().split(',')]

controller = []
for i in range(50):
    controller.append(intcode(data[:]))
    controller[i].add_input(i)
    
nat = []


def run():
    p1 = 0
    last_y = 0
    while 1:
        for i in range(50):
            try:
                while 1:
                    if len(controller[i].inputs) < 2:
                        controller[i].add_input(-1)

                    dest_address = controller[i].next_out()

                    if dest_address == None:
                        break

                    X = controller[i].next_out()
                    Y = controller[i].next_out()
                    #print(i, dest_address, X, Y)

                    if dest_address == 255: # 18513
                        if p1 == 0:
                            p1 = Y
                        nat = [X, Y]

                    else:
                        controller[dest_address].add_input(X)
                        controller[dest_address].add_input(Y)

            except StopIteration:
                print('stop')

            idle = True
            for n in range(50):
                if len(controller[n].inputs) != 0:
                    idle = False

            if idle:
                controller[0].add_input(nat[0])
                controller[0].add_input(nat[1])
                if last_y == nat[1]:
                    return p1, last_y
                last_y = nat[1]
                break

print(run())
print(str(datetime.now() - startTime)[:-3])

(18513, 13286)
0:00:01.235
